In [ ]:
using BenchmarkTools
using Revise

In [ ]:
using CSV
using DataFrames
using JSON

using DataIO
using GCTGMT
using Support

using FeatureSetEnrichment

In [ ]:
#
benchmarks_directory_path = "/Users/kwat/Downloads/score_and_set/"

#
benchmark_ = []

#
method_ = []

for m in keys(score_set_new(["a", "b"], [-1.0, 1.0], ["a"]; plot = false))

    push!(method_, string(m, " extreme"), string(m, " area"))

end

method_ = String.(method_)

n_method = length(method_)

#
benchmark_x_method_rows = []

;

In [ ]:
for name in readdir(benchmarks_directory_path)
    
    #
    if name[1] == '.'

        continue

    end

    println(name)

    #
    benchmark_directory_path = joinpath(benchmarks_directory_path, name)

    element_, score_ =
        eachcol(read_data(joinpath(benchmark_directory_path, "gene_x_score.tsv")))

    json_dict = convert(
        Dict{String,Vector{String}},
        JSON.parse(open(joinpath(benchmark_directory_path, "gene_set.json"))),
    )

    #
    set_to_element_ = read_gmt(
        map(
            gmt_path -> replace(gmt_path, ".." => "/Users/kwat/Downloads/"),
            json_dict["gene_sets_tested"],
        ),
    )

    set_ = sort(collect(keys(set_to_element_)))

    n_set = length(set_)

    #
    set_to_method_to_result = score_set_new(element_, score_, set_to_element_)

    #
    set_x_method = Matrix(undef, n_set, n_method)

    for (set_i, (set, method_to_result)) in enumerate(set_to_method_to_result)

        set_x_method_row = []

        for result in values(method_to_result)

            append!(set_x_method_row, result[2:3])

        end

        set_x_method[set_i, :] = set_x_method_row

    end

    #
    for set in json_dict["gene_sets_positive"]

        #
        print("    ")

        if in(set, set_)

            println(set)

        else

            println(set, " (missing)")

            continue

        end

        #
        push!(benchmark_, string(name, '.', set))

        #
        benchmark_x_method_row = []

        for (method_i, set_score_) in enumerate(eachcol(set_x_method))

            set_score_, set_ = sort_like((Float64.(set_score_), set_))

            push!(benchmark_x_method_row, findfirst(set_ .== set) / n_set)

        end

        push!(benchmark_x_method_rows, Float64.(benchmark_x_method_row))

    end
    
end


In [ ]:
benchmark_x_method = Matrix(undef, length(benchmark_x_method_rows), n_method)

for (i, v) in enumerate(benchmark_x_method_rows)

    benchmark_x_method[i, :] = Float64.(v)

end

benchmark_x_method = DataFrame(benchmark_x_method, method_)

In [ ]:
benchmark_x_method_path = "benchmark_x_method.tsv"

In [ ]:
CSV.write(benchmark_x_method_path, benchmark_x_method)

In [ ]:
benchmark_x_method = read_data(benchmark_x_method_path)

In [ ]:
sort_like((median.(eachcol(benchmark_x_method)), names(benchmark_x_method)))[2]